In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

In [12]:
DEVICE_DATA_PATH = "./data/"

In [18]:
df = pd.read_excel(DEVICE_DATA_PATH + "Pulse switching data_SET.xlsx", skiprows=7, usecols="C:K")
# Change the name to "Pulse switching data_SET.xlsx" for plotting surface plots of SET pulses

In [19]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.interpolate import griddata

# Function to generate surface from points
def generate_surface(points):
    x = points[:, 0]
    y = points[:, 1]
    z = 1*points[:, 2]
    
    grid_x, grid_y = np.mgrid[min(x):max(x):50j, min(y):max(y):50j]
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')
    
    return grid_x, grid_y, grid_z

data = {
    "10 ns":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 ns'], axis=1)), axis=1),
    "1 ms":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['1 ms'], axis=1)), axis=1),
    "1 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['1 μs'], axis=1)), axis=1),
    "10 ms":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 ms'], axis=1)), axis=1),
    "10 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 μs'], axis=1)), axis=1),
    "100 ns":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['100 ns'], axis=1)), axis=1),
    "100 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['100 μs'], axis=1)), axis=1)
    
}
if np.sum(df['V']<0)>0:
    for key in data.keys():
        data[key][:,2] *=-1
min_x, min_y, min_z = float('inf'), float('inf'), float('inf')
max_x, max_y, max_z = float('-inf'), float('-inf'), float('-inf')

for points in data.values():
    min_x = min(min_x, np.min(points[:, 0]))
    max_x = max(max_x, np.max(points[:, 0]))
    min_y = min(min_y, np.min(points[:, 1]))
    max_y = max(max_y, np.max(points[:, 1]))
    min_z = min(min_z, np.min(points[:, 2]))
    max_z = max(max_z, np.max(points[:, 2]))
fig = make_subplots(
    rows=3, cols=3,  
    specs=[
        [{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}],
        [{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}],
        [{'type': 'surface'}, None, None] 
    ],
    subplot_titles=("10ns", "1ms", "1 μs", "10 ms", "10 μs", "100 ns", "100 μs")
)

for i, key in enumerate(data):
    grid_x, grid_y, grid_z = generate_surface(data[key])
    
    row = i // 3 + 1
    col = i % 3 + 1
    
    fig.add_trace(go.Surface(
        z=grid_z, x=grid_x, y=grid_y,
        surfacecolor=np.full_like(grid_z, 5),
        colorscale="viridis",
        showscale=False  
    ), row=row, col=col)
shared_scene = dict(
    xaxis=dict(range=[min_x, max_x], title="Conductance (μs)"), 
    yaxis=dict(range=[min_y, max_y], title="Voltage (V)"), 
    zaxis=dict(range=[min_z, max_z], title="Delta G")  
)
fig.update_layout(
    title="G vs Vt vs delta G graph over different Ts",
    height=1200, width=1200,
    scene=shared_scene,
    scene2=shared_scene,
    scene3=shared_scene,
    scene4=shared_scene,
    scene5=shared_scene,
    scene6=shared_scene,
    scene7=shared_scene
)

fig.show()


Analyzed surface plots of **SET** and **RESET** pulses

In [20]:
shared_scene

{'xaxis': {'range': [5e-07, 7.5e-06], 'title': 'Conductance (μs)'},
 'yaxis': {'range': [1.6, 2.6], 'title': 'Voltage (V)'},
 'zaxis': {'range': [0.845236955786545, 10.800413702638615],
  'title': 'Delta G'}}

In [31]:
from scipy.interpolate import SmoothBivariateSpline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Sample data (replace with your actual data)
# key = list(data.keys())[2]
key = "1 us"
G0 = data[key][:,0]
V = data[key][:, 1]
deltaG_G0 = data[key][:, 2]

# Fit a cubic spline surface
spline = SmoothBivariateSpline(G0, V, deltaG_G0, kx=5, ky=5)

# Generate a grid over which to evaluate the spline
G0_grid, V_grid = np.meshgrid(np.linspace(G0.min(), G0.max(), 100),
                              np.linspace(V.min(), V.max(), 100))
Z_grid = spline.ev(G0_grid.ravel(), V_grid.ravel()).reshape(100, 100)

# Create the surface plot
surface = go.Surface(x=G0_grid, y=V_grid, z=Z_grid, colorscale='Viridis', opacity=0.7)

# Create the scatter plot for the original data points
scatter = go.Scatter3d(x=G0, y=V, z=deltaG_G0, mode='markers', 
                       marker=dict(size=5, color='red'), name='Data Points')

# Set up the layout of the plot
layout = go.Layout(scene=dict(
                    xaxis_title='G0',
                    yaxis_title='V',
                    zaxis_title="deltaG/G0"),
                    title=f"Spline fit for {key}")

# Create the figure
fig = go.Figure(data=[surface, scatter], layout=layout)

# Show the plot
fig.show()

In [22]:
np.sum(Z_grid<=0)

0

In [23]:
def get_spline(G, V, delta_G):
    return SmoothBivariateSpline(G, V, delta_G, kx=5, ky=5)
splines = {key:get_spline(data[key][:, 0], data[key][:, 1], data[key][:, 2]) for key in data.keys()}

In [24]:
splines

{'10 ns': <scipy.interpolate._fitpack2.SmoothBivariateSpline at 0x192220c9850>,
 '1 ms': <scipy.interpolate._fitpack2.SmoothBivariateSpline at 0x192220cb2d0>,
 '1 us': <scipy.interpolate._fitpack2.SmoothBivariateSpline at 0x192220cadd0>,
 '10 ms': <scipy.interpolate._fitpack2.SmoothBivariateSpline at 0x192220f8a10>,
 '10 us': <scipy.interpolate._fitpack2.SmoothBivariateSpline at 0x192220faf10>,
 '100 ns': <scipy.interpolate._fitpack2.SmoothBivariateSpline at 0x192220f83d0>,
 '100 us': <scipy.interpolate._fitpack2.SmoothBivariateSpline at 0x192220f9190>}

In [25]:
with open('data/set_splines.pkl', 'wb') as f:
    pickle.dump(splines, f)

In [26]:
with open('data/splines.pkl', 'rb') as f:
    loaded_splines = pickle.load(f)


In [27]:
def piecewise_spline_representation(spline):
    knots = spline.get_knots()
    coeffs = spline.get_coeffs()
    piecewise_repr = "Piecewise Spline Representation:\n"
    for i in range(len(knots[0]) - 1):
        piecewise_repr += f"Interval {i}: [{knots[0][i]}, {knots[0][i + 1]}] with coefficients: {coeffs[i]}\n"
    return piecewise_repr

# Display the piecewise representation
print(piecewise_spline_representation(spline))

Piecewise Spline Representation:
Interval 0: [5e-07, 5e-07] with coefficients: 1.352160969832068
Interval 1: [5e-07, 5e-07] with coefficients: 0.24872702696138954
Interval 2: [5e-07, 5e-07] with coefficients: 2.3707754597673336
Interval 3: [5e-07, 5e-07] with coefficients: -0.33351803087774334
Interval 4: [5e-07, 5e-07] with coefficients: 1.6469192145097924
Interval 5: [5e-07, 7.5e-06] with coefficients: 0.9876898529009279
Interval 6: [7.5e-06, 7.5e-06] with coefficients: 2.0584917618172214
Interval 7: [7.5e-06, 7.5e-06] with coefficients: -0.57347123555716
Interval 8: [7.5e-06, 7.5e-06] with coefficients: 0.867884073198879
Interval 9: [7.5e-06, 7.5e-06] with coefficients: 6.5864444656838925
Interval 10: [7.5e-06, 7.5e-06] with coefficients: -3.31553663351384



In [28]:
# df = pd.read_excel(DEVICE_DATA_PATH + "Pulse switching data_SET.xlsx", skiprows=7, usecols="C:K")
# set_data = {
#     "10 ns":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 ns'], axis=1)), axis=1),
#     "1 ms":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['1 ms'], axis=1)), axis=1),
#     "1 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['1 μs'], axis=1)), axis=1),
#     "10 ms":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 ms'], axis=1)), axis=1),
#     "10 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 μs'], axis=1)), axis=1),
#     "100 ns":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['100 ns'], axis=1)), axis=1),
#     "100 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['100 μs'], axis=1)), axis=1)
    
# }
# df = pd.read_excel(DEVICE_DATA_PATH + "Pulse switching data_RESET.xlsx", skiprows=7, usecols="C:K")
# if np.sum(df['V']<0)>0:
#     for key in data.keys():
#         data[key][:,2] *=-1
# reset_data = {
#     "10 ns":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 ns'], axis=1)), axis=1),
#     "1 ms":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['1 ms'], axis=1)), axis=1),
#     "1 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['1 μs'], axis=1)), axis=1),
#     "10 ms":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 ms'], axis=1)), axis=1),
#     "10 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['10 μs'], axis=1)), axis=1),
#     "100 ns":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['100 ns'], axis=1)), axis=1),
#     "100 us":np.concatenate((np.expand_dims( df['STATE'], axis=1), np.expand_dims(df['V'], axis=1), np.expand_dims(df['100 μs'], axis=1)), axis=1)
    
# }
with open('data/reset_splines.pkl', 'rb') as f:
    reset_splines = pickle.load(f)
with open('data/set_splines.pkl', 'rb') as f:
    set_splines = pickle.load(f)

In [29]:
for g0 in np.arange(5e-7, 6e-6, 5e-8):
    print((set_splines['10 us'].ev(g0, +2.5)))

1.492555870248918
1.4775266321229263
1.4631743144304776
1.4494650206494335
1.4363660218385557
1.4238457381762002
1.4118737204989948
1.4004206318405177
1.389458228969989
1.3789593439309495
1.3688978655799418
1.3592487211251971
1.3499878576653173
1.3410922237279554
1.3325397508085015
1.324309334908767
1.316380818075662
1.3087349699398805
1.3013534692545907
1.2942188854341081
1.2873146600925796
1.2806250885826764
1.2741353015342642
1.2678312463930936
1.2616996689594815
1.2557280949269956
1.249904811421133
1.2442188485380081
1.2386599608830338
1.2332186091096031
1.2278859414577719
1.2226537752929492
1.2175145786445694
1.2124614517447814
1.2074881085671305
1.2025888583652427
1.1977585872115053
1.192992739535751
1.1882872996639422
1.1836387733568512
1.179044169348746
1.1745009808860738
1.170007167266138
1.1655611353757873
1.1611617212301018
1.156808171511065
1.1525001251062543
1.1482375946475258
1.144020948049691
1.139850890049205
1.1357284437428476
1.1316549321264058
1.1276319596333562
1.12

In [30]:
def map_weights_to_conductance(weight, min_weight, max_weight, min_conductance, max_conductance):
            # Linear interpolation from weight to conductance
    normalized_weights = (weight - min_weight) / (max_weight - min_weight)
    mapped_conductance = min_conductance + normalized_weights * (max_conductance - min_conductance)
    return mapped_conductance

def new_conductance(G0, grad):
    if grad >= 0:
        return G0*(set_splines['10 us'].ev(G0, 2.5) + 1)
    else:
        return G0*(reset_splines['10 us'].ev(G0, -2.5) + 1)